# Draft notebook for the application

## General imports

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
import os

## Test book content loader

In [16]:
import loaders.book_content_loader as bcl

In [21]:
search_query = "Martin Eden"
res_search = bcl.search_books(search_query)

In [22]:
url_archive = bcl.get_book_archive_page(res_search)

In [23]:
book_text = bcl.fetch_book_text(url_archive)

Book found with length: 930190 characters
es.  She 
knew  neither  her  own  potencies,  nor  the  potencies  of  the 
world;  and  the  deeps  of  life  were  to  her  seas  of  illusion. 
The  conjugal  affection  of  her  father  and  moth


In [24]:
path_book_content = bcl.write_book_to_file(book_text, search_query)

Book text written to documents/Martin_Eden.txt


In [25]:
path_book_content

'documents/Martin_Eden.txt'

## RAG with HuggingFace

In [ ]:
import retrieval.rag_retriever as rr

In [32]:
if not os.path.exists("documents"):
    os.makedirs("documents")

with open("documents/sample.txt", "w") as f:
    f.write("""
Nicolas Cotoni is a french computer engineer. From Lyon he was graduated the engineering school named EPITA.
""")

# Initialize the RAG system
model_name = "Qwen/Qwen2.5-3B-Instruct" if torch.cuda.is_available() else "HuggingFaceTB/SmolLM2-135M"
rag = rr.LocalRAGSystem("documents/sample.txt", model_name=model_name)
rag.initialize()
answer_tag = "**Answer:**"

# Example query
result = rag.query("Who is Nicolas Cotoni?")
print("\nQuestion: Who is Nicolas Cotoni?")
print("\nAnswer:", result["result"][result["result"].find(answer_tag) + len(answer_tag):])


Loaded 1 document chunks
Vector store created successfully
Loading model: HuggingFaceTB/SmolLM2-135M


Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


RAG system initialized successfully!

Question: Who is Nicolas Cotoni?

Answer: 

Nicolas Cotoni is a French computer engineer who works at EPITA. He has been working there for 10 years and is now retired from his job.

He worked as an engineer at EPITA for 25 years before he started working at the company.

His main role is to design software that can be used by people all over the world.

The book contains many details about him including his work experience, education, hobbies etc.

This character will be discussed in detail later in the course.

## Chapter 3: The Story of the Book

Chapter 4: The Story of the Book

### Chapter 4.1: The Story of the Book

In this chapter we are going to talk about the story of the book.

We’re going to discuss how the author created the story of the book.

What makes the story so interesting?

How does the author create suspense?

Why do you think the author chose to write this book?

Is it possible to read this book again?

If yes, what would you l